In [ ]:
from indic_asr_onnx import IndicTranscriber
from anuvaad_rev import IndicTranslator
import gradio as gr
from gradio_client import Client
from gradio_client import handle_file
import ffmpeg

In [ ]:
transcriber = IndicTranscriber()
translator = IndicTranslator()

In [ ]:
token="your_hugging_face_token"

In [ ]:
voicecln= Client("ai4bharat/IndicF5",hf_token=token)

In [ ]:
langauges=["Assamese","Bengali","Gujarati","Hindi","Kannada","Malayalam","Marathi","Odia","Punjabi","Tamil","Telugu"]


In [ ]:
lang_to_code = {"Assamese": "as","Bengali": "bn","Gujarati": "gu","Hindi": "hi","Kannada": "kn","Malayalam": "ml","Marathi": "mr",
    "Odia": "or","Punjabi": "pa","Tamil": "ta","Telugu": "te"}


In [ ]:
custom_css = """
body {
    background: linear-gradient(135deg, #667eea, #764ba2);
}
"""

In [ ]:
def clonevoice(audio,clang,tlang):
  outfile="paudio.wav"
  if audio:
    ffmpeg.input(audio).output(outfile, ar=16000, ac=1, acodec="pcm_s16le").overwrite_output().run(quiet=True)



    text = transcriber.transcribe_ctc(outfile, lang_to_code[clang])

    translation = translator.translate(text, lang_to_code[tlang])

    caudio=voicecln.predict(text=translation,ref_audio=handle_file(outfile),ref_text=text,api_name="/synthesize_speech")

    return text,translation,caudio
  return "no audio provided",None,None






In [ ]:
with gr.Blocks(css=custom_css, theme=gr.themes.Soft()) as demo:

    with gr.Tab("clone your voice"):
        with gr.Row():
            audio_input = gr.Audio(
                sources=["microphone", "upload"],
                type="filepath",          # returns temp file path
                label="Upload or Record Audio"
            )
            clang = gr.Dropdown(langauges, label="Current Language")
            tlang = gr.Dropdown(langauges, label="Target Language")


        submit_btn = gr.Button("Submit")

        output_text = gr.Textbox(label="Result")
        output_text2 = gr.Textbox(label="Results")
        output_audio = gr.Audio(label="Output Audio")

        submit_btn.click(
            fn=clonevoice,
            inputs=[audio_input, clang, tlang],
            outputs=[output_text, output_text2, output_audio]
        )


demo.launch(
    server_name="0.0.0.0",
    server_port=5000,
    debug=True
)
